In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import csv
import scipy
import astropy
from astropy.timeseries import LombScargle

# define a function to calculate the minus_fft when given a sequence and a list of windows
def minus_fft_calculator(sequence, windows, cosine_taper_length=50):
    """
    sequence: the sequence of the signal
    windows: a list of windows which has the start and end point of the window
    cosine_taper_length: the length of the cosine taper
    """
    
    taper = np.cos(np.linspace(0, np.pi, int(cosine_taper_length*2)))
    window_summation = np.zeros(len(sequence))
    for window in windows:
        start_dx = window[0]
        end_dx = window[1]
        temp_window = np.zeros(len(sequence))
        temp_window[start_dx:end_dx] = 1
        # make the edge smooth with a cosine window
        #print(start_dx, end_dx, cosine_taper_length, len(temp_window), len(taper))
        try:
            temp_window[start_dx-cosine_taper_length:start_dx] = temp_window[start_dx-cosine_taper_length:start_dx]*taper[0:cosine_taper_length]
            temp_window[end_dx:end_dx+cosine_taper_length] = temp_window[end_dx:end_dx+cosine_taper_length]*taper[cosine_taper_length:]
        except:
            continue
        window_summation = window_summation + temp_window
    # apply the window to the signal
    sequence_windowed = sequence*window_summation

    # apply hann window to the windowed signal
    sequence_windowed = sequence_windowed*scipy.signal.windows.hann(len(sequence))
    sequence_windowed_fft = np.fft.fft(sequence_windowed)
    # apply hann window to the original signal
    sequence = sequence*scipy.signal.windows.hann(len(sequence))

    sequence_fft = np.fft.fft(sequence)

    minus_fft = sequence_fft - sequence_windowed_fft
    
    return minus_fft

def remove_detected_windows(time_array, data_array, detected_windows):
    """
    Removes data corresponding to abnormal windows from the time and observation arrays.
    
    Parameters:
        time_array (np.ndarray): Array of time values.
        data_array (np.ndarray): Array of observation values.
        detected_windows (list of tuples): List of abnormal windows, each defined by a tuple (start_index, end_index).
    
    Returns:
        tuple: Filtered time_array and data_array with abnormal data removed.
    """
    if len(time_array) != len(data_array):
        raise ValueError("time_array and data_array must have the same length.")
    
    # Create a mask of valid data points
    valid_mask = np.ones(len(time_array), dtype=bool)
    
    # Mark indices in abnormal windows as invalid
    for start_idx, end_idx in detected_windows:
        valid_mask[start_idx:end_idx + 1] = False
    
    # Apply the mask to both arrays
    filtered_time = time_array[valid_mask]
    filtered_observation = data_array[valid_mask]
    
    return filtered_time, filtered_observation

def plot_seismogram_with_gaps(filtered_t, filtered_seismogram, gap_threshold=6.625):
    # Detect gaps based on the filtered time array
    segments = []
    current_segment = [0]

    for i in range(1, len(filtered_t)):
        if filtered_t[i] - filtered_t[i - 1] > gap_threshold:
            current_segment.append(i - 1)
            segments.append(current_segment)
            current_segment = [i]
    current_segment.append(len(filtered_t) - 1)
    segments.append(current_segment)

    for segment in segments:
        start, end = segment
        plt.plot(
            filtered_t[start:end + 1],
            filtered_seismogram[start:end + 1],
            color='k'
        )
    return

def merge_window(windows):
    windows.sort(key=lambda x: x[0])

    merged_windows = [windows[0]]

    for current_start, current_end in windows[1:]:
        last_merged_start, last_merged_end = merged_windows[-1]

        # Check if the current window overlaps with the last merged window
        if current_start <= last_merged_end:
            # Merge the windows by extending the end time of the last merged window
            merged_windows[-1] = [last_merged_start, max(last_merged_end, current_end)]
        else:
            # No overlap, add the current window to the merged list
            merged_windows.append([current_start, current_end])

    # Update the windows list
    windows = merged_windows

    return windows

In [2]:
npy_name_list = ['EV_13_S4B_IMP.npy',
                 'EV_14_S4B_IMP.npy',
                 'EV_15_S4B_IMP.npy',
                 'EV_16_S4B_IMP.npy',
                 'EV_17_S4B_IMP.npy']


In [3]:
for npy_name in npy_name_list:
    res_dict = np.load('./{}'.format(npy_name), allow_pickle=True)[()]
    dt = 1.0/6.625
    for key in res_dict.keys():
        print(key)
        key_split = key.split('_')
        if key_split[0] == 'Overall':
            continue
        if 'S14' in key:
            continue

        data = res_dict[key]['trace'].data[:].astype(np.float64)
        print('On {}'.format(key))
        windows = res_dict[key]['windows']
        time_array = np.arange(0, len(data)*dt, dt)
        detected_windows = windows

        trace = res_dict[key]['trace']
        starttime = trace.stats['starttime']
        endtime = trace.stats['endtime']
        
        if windows[0][0] < 0:
            windows[0][0] = 0

        muted_data_fft = minus_fft_calculator(data, detected_windows)

        filtered_t, filtered_seismogram = remove_detected_windows(time_array, data, detected_windows)
        filtered_mean = np.mean(filtered_seismogram)
        filtered_seismogram -= filtered_mean
        frequency, power = LombScargle(filtered_t, filtered_seismogram).autopower(method='fastchi2',minimum_frequency=1e-5,maximum_frequency=1e-2,samples_per_peak=10)

        plt.figure(figsize=(20, 24))
        plt.subplot(6, 1, 1)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.plot(time_array, data, color = 'k')
        for window in detected_windows:
            plt.axvspan(window[0]*dt, window[1]*dt, color='pink', alpha=0.5, ymin=0.2, ymax=0.8)
        plt.axvspan(0,0,color='pink',alpha=0.5,label='AI Detected Windows', ymin=0.2, ymax=0.8)
        plt.legend(fontsize=16, loc='upper right')
        polluted_rate = sum([window[1]-window[0] for window in detected_windows])/len(data)
        #plt.title('Raw Apollo Data with {:.0f}% Detected as Disturbances'.format(polluted_rate*100), fontsize=20)
        #plt.xlabel('Time (s)', fontsize=20)
        title_str = '{}   {}'.format('XA.S12.MHZ', starttime.strftime('%Y-%m-%d %H:%M:%S'))
        plt.text(0.5, 0.975, '{:}\n{:.0f}% Detected as Disturbances'.format(title_str, polluted_rate*100), transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
        plt.xlabel('Time (hours)', fontsize=20)
        # a tick per 10 hour
        plt.xticks(np.arange(0, time_array[-1]+1, 36000), (np.arange(0, time_array[-1]+1, 36000)/3600).astype(int))
        plt.ylabel('DU', fontsize=20)
        plt.xlim(0, time_array[-1])
        plt.ylim(np.min(data)-100, np.max(data)+100)
        plt.text(-0.05, 1.00, '(a)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')

        plt.subplot(6, 1, 2)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        # fft of the original signal
        freqs = np.fft.fftfreq(len(data), dt)[:len(data)//2]
        plt.plot(freqs, np.abs(np.fft.fft(data))[:len(data)//2], color='k')

        fft_power = np.abs(np.fft.fft(data))[:len(data)//2]
        x_index = np.where((freqs>1e-3) & (freqs<1e-2))
        y_max = np.max(fft_power[x_index])
        y_min = np.min(fft_power[x_index])
        plt.ylim(y_min*0.8, y_max*1.4)

        #plt.title('Spectrum of Raw Apollo Data', fontsize=20)
        plt.text(0.5, 0.975, 'Spectrum of Raw Apollo Data', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
        plt.xlabel('Frequency (Hz)', fontsize=20)
        plt.ylabel('Amplitude', fontsize=20)
        plt.xlim(1e-3, 1e-2)
        plt.text(-0.05, 1.00, '(b)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')

        # No LPS at after because its lunar daytime by then
        if key_split[0] == 'Before':
            plt.axvspan(1/540, 1/570, color='orange', alpha=0.5, label='base frequency of LPS')
            plt.text(1/540, y_max*0.90, '540-570 s', fontsize=18)

            plt.axvspan(1/270, 1/285, color='coral', alpha=0.5, label='x2 frequency of LPS')
            plt.text(1/270, y_max*0.90, '270-285 s', fontsize=18)

            plt.axvspan(1/180, 1/190, color='lightcoral', alpha=0.5, label='x3 frequency of LPS')
            plt.text(1/180, y_max*0.90, '180-190 s', fontsize=18)
            plt.legend(fontsize=16, loc='upper right')
        #plt.legend(fontsize=16, loc='upper right')

        plt.subplot(6, 1, 3)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        # mute the data to zero in the detected windows
        muted_data_in_time_domain = data.copy()
        muted_data_in_time_domain -= filtered_mean

        for window in detected_windows:
            muted_data_in_time_domain[window[0]:window[1]] = 0

        plt.plot(time_array, muted_data_in_time_domain, color = 'k')
        #plt.title('Muted Apollo Data in Time-Domain', fontsize=20)
        plt.text(0.5, 0.975, 'Muted Apollo Data in Time-Domain', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
        #plt.xlabel('Time (s)', fontsize=20)
        plt.xlabel('Time (hours)', fontsize=20)
        # a tick per 10 hour
        plt.xticks(np.arange(0, time_array[-1]+1, 36000), (np.arange(0, time_array[-1]+1, 36000)/3600).astype(int))
        plt.ylabel('DU', fontsize=20)
        plt.xlim(0, time_array[-1])
        plt.text(-0.05, 1.00, '(c)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')

        plt.subplot(6, 1, 4)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        # fft of the muted signal
        plt.plot(freqs, np.abs(np.fft.fft(muted_data_in_time_domain))[:len(data)//2], color='k')
        #plt.title('Spectrum of Muted Apollo Data', fontsize=20)
        plt.text(0.5, 0.975, 'Spectrum of Muted Apollo Data', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
        plt.xlabel('Frequency (Hz)', fontsize=20)
        plt.ylabel('Amplitude', fontsize=20)
        plt.xlim(1e-3, 1e-2)
        # use the amplitude within this range
        fft_power = np.abs(np.fft.fft(muted_data_in_time_domain))[:len(data)//2]
        x_index = np.where((freqs>1e-3) & (freqs<1e-2))
        y_max = np.max(fft_power[x_index])
        y_min = np.min(fft_power[x_index])
        plt.ylim(y_min*0.8, y_max*1.2)
        #plt.ylim(0,1.25e9)
        plt.text(-0.05, 1.00, '(d)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')
        """
        plt.axvline(1/470, color='pink', linestyle='--')
        plt.text(1/470, y_max*0.90, '470 s', fontsize=16)
        
        plt.axvline(1/570, color='pink', linestyle='--')
        plt.text(1/570, y_max*0.90, '570 s', fontsize=16)

        plt.axvline(1/235, color='pink', linestyle='--')
        plt.text(1/235, y_max*0.90, '235 s', fontsize=16)

        plt.axvline(1/285, color='pink', linestyle='--')
        plt.text(1/285, y_max*0.90, '285 s', fontsize=16)
        """
        if key_split[0] == 'Before':
            plt.axvspan(1/540, 1/570, color='orange', alpha=0.5, label='base frequency of LPS')
            plt.text(1/540, y_max*0.90, '540-570 s', fontsize=18)

            plt.axvspan(1/270, 1/285, color='coral', alpha=0.5, label='x2 frequency of LPS')
            plt.text(1/270, y_max*0.90, '270-285 s', fontsize=18)

            plt.axvspan(1/180, 1/190, color='lightcoral', alpha=0.5, label='x3 frequency of LPS')
            plt.text(1/180, y_max*0.90, '180-190 s', fontsize=18)

        #plt.legend(fontsize=16, loc='upper right')
        
        plt.subplot(6, 1, 5)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        # the gapped data
        #plt.plot(filtered_t, filtered_seismogram, color = 'k')
        plot_seismogram_with_gaps(filtered_t, filtered_seismogram)
        # set the boundary of the plot to be above the detected windows

        #plt.title('Gapped Apollo Data', fontsize=20)
        plt.text(0.5, 0.975, 'Gapped Apollo Data', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
        #plt.xlabel('Time (s)', fontsize=20)
        plt.xlabel('Time (hours)', fontsize=20)
        # a tick per 10 hour
        plt.xticks(np.arange(0, time_array[-1]+1, 36000), (np.arange(0, time_array[-1]+1, 36000)/3600).astype(int))
        
        plt.ylabel('DU', fontsize=20)
        plt.xlim(0, time_array[-1])
        plt.text(-0.05, 1.00, '(e)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')

        plt.subplot(6, 1, 6)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        # the Lomb-Scargle periodogram
        plt.plot(frequency, power, color='k')
        #plt.title('Lomb-Scargle Periodogram', fontsize=20)
        plt.text(0.5, 0.975, 'Lomb-Scargle Periodogram', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
        plt.xlabel('Frequency (Hz)', fontsize=20)
        plt.ylabel('Power', fontsize=20)
        plt.xlim(1e-3, 1e-2)
        x_index = np.where((frequency>1e-3) & (frequency<1e-2))
        y_max = np.max(power[x_index])
        y_min = np.min(power[x_index])
        plt.ylim(y_min*0.8, y_max*1.2)
        plt.text(-0.05, 1.00, '(f)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')
        if key_split[0] == 'Before':
            plt.axvspan(1/540, 1/570, color='orange', alpha=0.5, label='base frequency of LPS')
            plt.text(1/540, y_max*0.90, '540-570 s', fontsize=18)

            plt.axvspan(1/270, 1/285, color='coral', alpha=0.5, label='x2 frequency of LPS')
            plt.text(1/270, y_max*0.90, '270-285 s', fontsize=18)

            plt.axvspan(1/180, 1/190, color='lightcoral', alpha=0.5, label='x3 frequency of LPS')
            plt.text(1/180, y_max*0.90, '180-190 s', fontsize=18)

            
        plt.tight_layout()
        
        plt.savefig('./Figure_6_7_S23_30_'+key+'.png', dpi=600)
        print(key+'.png'+' saved')
        plt.close()

Before_1970.105.S12.MHZ
On Before_1970.105.S12.MHZ
Before_1970.105.S12.MHZ.png saved
After_1970.105.S12.MHZ
On After_1970.105.S12.MHZ
After_1970.105.S12.MHZ.png saved
Before_1971.35.S12.MHZ
On Before_1971.35.S12.MHZ
Before_1971.35.S12.MHZ.png saved
After_1971.35.S12.MHZ
On After_1971.35.S12.MHZ
After_1971.35.S12.MHZ.png saved
Before_1971.210.S12.MHZ
On Before_1971.210.S12.MHZ
Before_1971.210.S12.MHZ.png saved
After_1971.210.S12.MHZ
On After_1971.210.S12.MHZ
After_1971.210.S12.MHZ.png saved
Before_1972.110.S12.MHZ
On Before_1972.110.S12.MHZ
Before_1972.110.S12.MHZ.png saved
After_1972.110.S12.MHZ
On After_1972.110.S12.MHZ
After_1972.110.S12.MHZ.png saved
Before_1972.345.S12.MHZ
On Before_1972.345.S12.MHZ
Before_1972.345.S12.MHZ.png saved
After_1972.345.S12.MHZ
On After_1972.345.S12.MHZ
After_1972.345.S12.MHZ.png saved
